In [2]:
from backtrader import plot
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from backtrader.utils import num2date,date2num
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go
import plotly.express as px
import rba_tools.backtest.backtrader_extensions.plot as rbsplot
import importlib


In [10]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [11]:
cerebro = bt.Cerebro(runonce=False)

# Add a strategy
cerebro.addstrategy(MaCrossStrategy)
#cerebro.addstrategy(rbsstrat.ConsecutiveBarsTest)
#cerebro.addstrategy(rbsstrat.StopLimitEntryStrategy)

#get a datapuller
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()

#symbol and date range
symbol = 'ETH/USD'
from_date = '1/1/20'
to_date = '6/30/20'

# Get a pandas dataframe
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)

# Pass it to the backtrader datafeed and add it to the cerebro
data = bt.feeds.PandasData(dataname=dataframe,
                            nocase=True,
                            )
#cerebro.adddata(data)
cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=240)

cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=1440)

# Run over everything
back = cerebro.run()

# Plot the result
#myp = cerebro.plot(numfigs=1, style='bar')

In [12]:
strategy = back[0]
from backtrader import plot
plotter = plot.Plot()
plotter

In [13]:
plotter.sortdataindicators(strategy)

In [14]:
vars(plotter)

{'params': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x21efd4fbfd0>,
 'p': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x21efd4fbfd0>,
 'dplotstop': [<backtrader.observers.broker.Broker at 0x21efd521e80>,
 'dplotsup': defaultdict(list, {}),
 'dplotsdown': defaultdict(list,
             {<backtrader.indicators.sma.SMA at 0x21efd5139d0>: [<backtrader.indicators.crossover.CrossOver at 0x21efd48b520>]}),
 'dplotsover': defaultdict(list,
             {<backtrader.feeds.pandafeed.PandasData at 0x21efd480c40>: [<backtrader.observers.buysell.BuySell at 0x21efd5187f0>,
              <backtrader.feed.DataClone at 0x21efd4b8a60>: [<backtrader.observers.buysell.BuySell at 0x21efd4f45e0>]})}